In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 14:08:08.539544


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we need to define the parameters, decision variables, objective function, and constraints.

**Parameters:**
- \(Q\) is the total required amount of units, which is 239,600,480.
- \(C_{A,s}\) is the setup cost for Vendor A, which is $3855.34.
- \(C_{A,u}\) is the unit cost per thousand units for Vendor A, which is $61.150.
- \(Q_{A,max}\) is the maximum supply capacity for Vendor A, which is 33 million units.
- \(C_{B,s1}\), \(C_{B,s2}\), \(C_{B,s3}\), and \(C_{B,s4}\) are setup costs for Vendor B for different quantity ranges.
- \(C_{B,u1}\), \(C_{B,u2}\), \(C_{B,u3}\), and \(C_{B,u4}\) are unit costs per thousand units for Vendor B for different quantity ranges.
- \(Q_{B1,min}\) and \(Q_{B1,max}\), \(Q_{B2,min}\) and \(Q_{B2,max}\), \(Q_{B3,min}\) and \(Q_{B3,max}\), \(Q_{B4,min}\) and \(Q_{B4,max}\) are the minimum and maximum supply limits for Vendor B for different quantity ranges.
- \(C_{C,s}\) is the setup cost for Vendor C, which is $13,456.00.
- \(C_{C,u}\) is the unit cost per thousand units for Vendor C, which is $62.019.
- \(Q_{C,max}\) is the maximum supply capacity for Vendor C, which is 165.6 million units.
- \(C_{D,s}\) is the setup cost for Vendor D, which is $6,583.98.
- \(C_{D,u}\) is the unit cost per thousand units for Vendor D, which is $72.488.
- \(Q_{D,max}\) is the maximum supply capacity for Vendor D, which is 12 million units.
- \(C_{E,s1}\) and \(C_{E,s2}\) are setup costs for Vendor E for different quantity ranges.
- \(C_{E,u1}\) and \(C_{E,u2}\) are unit costs per thousand units for Vendor E for different quantity ranges.
- \(Q_{E1,min}\) and \(Q_{E1,max}\), \(Q_{E2,min}\) and \(Q_{E2,max}\) are the minimum and maximum supply limits for Vendor E for different quantity ranges.
- \(d\) is the rate of decrease in unit cost for Vendor E when purchasing between 42,000,001 and 77 million units, which is 0.05%.

**Decision Variables:**
- \(x_A\) is the number of units to purchase from Vendor A.
- \(x_{B1}\), \(x_{B2}\), \(x_{B3}\), and \(x_{B4}\) are the number of units to purchase from Vendor B for different quantity ranges.
- \(x_C\) is the number of units to purchase from Vendor C.
- \(x_D\) is the number of units to purchase from Vendor D.
- \(x_{E1}\) and \(x_{E2}\) are the number of units to purchase from Vendor E for different quantity ranges.

**Objective Function:**
The objective is to minimize the total cost, which can be expressed as:

\[\text{minimize } Cost = C_{A,s} \frac{x_A}{1000} + C_{A,u} x_A + C_{B,s1} \frac{x_{B1}}{1000} + C_{B,u1} x_{B1} + C_{B,s2} \frac{x_{B2}}{1000} + C_{B,u2} x_{B2} + C_{B,s3} \frac{x_{B3}}{1000} + C_{B,u3} x_{B3} + C_{B,s4} \frac{x_{B4}}{1000} + C_{B,u4} x_{B4} + C_{C,s} \frac{x_C}{1000} + C_{C,u} x_C + C_{D,s} \frac{x_D}{1000} + C_{D,u} x_D + C_{E,s1} \frac{x_{E1}}{1000} + C_{E,u1} x_{E1} + C_{E,s2} \frac{x_{E2}}{1000} + C_{E,u2} x_{E2}\]

**Constraints:**
- \(x_A \leq Q_{A,max}\): Vendor A's supply constraint.
- \(x_{B1} \in \{0, Q_{B1,min}, Q_{B1,max}\}, x_{B2} \in \{0, Q_{B2,min}, Q_{B2,max}\}, x_{B3} \in \{0, Q_{B3,min}, Q_{B3,max}\}, x_{B4} \in \{0, Q_{B4,min}, Q_{B4,max}\}\): Vendor B's supply constraints for different quantity ranges.
- \(x_C \leq Q_{C,max}\): Vendor C's supply constraint.
- \(x_D \leq Q_{D,max}\): Vendor D's supply constraint.
- \(x_{E1} \in \{0, Q_{E1,min}, Q_{E1,max}\}, x_{E2} \in \{0, Q_{E2,min}, Q_{E2,max}\}\): Vendor E's supply constraints for different quantity ranges.
- \(x_A + x_{B1} + x_{B2} + x_{B3} + x_{B4} + x_C + x_D + x_{E1} + x_{E2} = Q\): Total supply constraint.
- \(x_{B2} \leq Q_{B1,max} - Q_{B1,min}\): Constraint to ensure that the quantity purchased from Vendor B in the second range does not exceed the difference between the maximum and minimum quantities in the first range.
- \(x_{B3} \leq Q_{B2,max} - Q_{B2,min}\): Constraint to ensure that the quantity purchased from Vendor B in the third range does not exceed the difference between the maximum and minimum quantities in the second range.
- \(x_{B4} \leq Q_{B3,max} - Q_{B3,min}\): Constraint to ensure that the quantity purchased from Vendor B in the fourth range does not exceed the difference between the maximum and minimum quantities in the third range.
- \(x_{E2} \leq Q_{E2,max} - Q_{E2,min}\): Constraint to ensure that the quantity purchased from Vendor E in the second range does not exceed the difference between the maximum and minimum quantities in the first range.

This is a complex optimization problem that can be solved using a variety of optimization techniques, such as linear programming or mixed-integer programming, depending on the specific problem characteristics and constraints.

In [7]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we need to define the parameters, decision variables, objective function, and constraints.

**Parameters:**
- \(Q\) is the total required amount of units, which is 239,600,480.
- \(C_{A,s}\) is the setup cost for Vendor A, which is $3855.34.
- \(C_{A,u}\) is the unit cost per thousand units for Vendor A, which is $61.150.
- \(Q_{A,max}\) is the maximum supply capacity for Vendor A, which is 33 million units.
- \(C_{B,s1}\), \(C_{B,s2}\), \(C_{B,s3}\), and \(C_{B,s4}\) are setup costs for Vendor B for different quantity ranges.
- \(C_{B,u1}\), \(C_{B,u2}\), \(C_{B,u3}\), and \(C_{B,u4}\) are unit costs per thousand units for Vendor B for different quantity ranges.
- \(Q_{B1,min}\) and \(Q_{B1,max}\), \(Q_{B2,min}\) and \(Q_{B2,max}\), \(Q_{B3,min}\) and \(Q_{B3,max}\), \(Q_{B4,min}\) and \(Q_{B4,max}\) are the minimum and maximum supply limits for Vendor B for different quantity ranges.
- \(C_{C,s}\) is the setup cost for 

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

To solve this problem using Pyomo, we first need to install the required packages. You can install Pyomo and other necessary packages using the following command:

```
!pip install pyomo
```

Now, we can write the Pyomo code for this optimization problem. Here's the code with some sample data for Vendor B and Vendor E's setup costs, unit costs, and supply limits:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
Q = 239600480

C_A_s = 3855.34
C_A_u = 61.150
Q_A_max = 33000000

C_B_s = [3987.26, 2887.65, 2312.94, 1967.37]
C_B_u = [63.143, 61.949, 61.356, 61.041]
Q_B_min = [0, 12000000, 30000000, 60000000]
Q_B_max = [12000000, 30000000, 60000000, 120000000]

C_C_s = 13456.00
C_C_u = 62.019
Q_C_max = 165600000

C_D_s = 6583.98
C_D_u = 72.488
Q_D_max = 12000000

C_E_s = [6088.20, 5205.72]
C_E_u = [67.825, 64.633]
Q_E_min = [0, 42000000]
Q_E_max = [42000000, 77000000]

d = 0.05 / 100

# Define the decision variables
model.x_A = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_A_max))
model.x_B = pyo.Var(4, within=pyo.Binary)
model.x_C = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_C_max))
model.x_D = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_D_max))
model.x_E = pyo.Var(2, within=pyo.Binary)

# Define the objective function
def cost_rule(model):
    return (
        C_A_s * model.x_A / 1000 + C_A_u * model.x_A +
        sum(C_B_s[i] * model.x_B[i] / 1000 + C_B_u[i] * model.x_B[i] for i in range(4)) +
        C_C_s * model.x_C / 1000 + C_C_u * model.x_C +
        C_D_s * model.x_D / 1000 + C_D_u * model.x_D +
        sum(C_E_s[i] * model.x_E[i] / 1000 + C_E_u[i] * model.x_E[i] for i in range(2))
    )
model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

# Define the constraints
model.constraints = pyo.ConstraintList()

model.constraints.add(model.x_A <= Q_A_max)

for i in range(4):
    model.constraints.add(model.x_B[i] * Q_B_min[i] <= model.x_B[i] * Q_B_max[i])

model.constraints.add(model.x_C <= Q_C_max)
model.constraints.add(model.x_D <= Q_D_max)

for i in range(2):
    model.constraints.add(model.x_E[i] * Q_E_min[i] <= model.x_E[i] * Q_E_max[i])

model.constraints.add(
    model.x_A + sum(model.x_B[i] * Q_B_max[i] for i in range(4)) +
    model.x_C + model.x_D + sum(model.x_E[i] * Q_E_max[i] for i in range(2)) == Q
)

model.constraints.add(model.x_B[1] <= Q_B_max[0] - Q_B_min[0])
model.constraints.add(model.x_B[2] <= Q_B_max[1] - Q_B_min[1])
model.constraints.add(model.x_B[3] <= Q_B_max[2] - Q_B_min[2])

model.constraints.add(model.x_E[1] <= Q_E_max[0] - Q_E_min[0])

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Optimal cost:', pyo.value(model.cost))
print('x_A:', pyo.value(model.x_A))
print('x_B:', [pyo.value(model.x_B[i]) for i in range(4)])
print('x_C:', pyo.value(model.x_C))
print('x_D:', pyo.value(model.x_D))
print('x_E:', [pyo.value(model.x_E[i]) for i in range(2)])
```

This code defines the optimization problem as described in the question, and then solves it using the GLPK solver. The results are then printed out, showing the optimal cost and the optimal values for the decision variables.

In [11]:
print(response_2_text)

To solve this problem using Pyomo, we first need to install the required packages. You can install Pyomo and other necessary packages using the following command:

```
!pip install pyomo
```

Now, we can write the Pyomo code for this optimization problem. Here's the code with some sample data for Vendor B and Vendor E's setup costs, unit costs, and supply limits:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
Q = 239600480

C_A_s = 3855.34
C_A_u = 61.150
Q_A_max = 33000000

C_B_s = [3987.26, 2887.65, 2312.94, 1967.37]
C_B_u = [63.143, 61.949, 61.356, 61.041]
Q_B_min = [0, 12000000, 30000000, 60000000]
Q_B_max = [12000000, 30000000, 60000000, 120000000]

C_C_s = 13456.00
C_C_u = 62.019
Q_C_max = 165600000

C_D_s = 6583.98
C_D_u = 72.488
Q_D_max = 12000000

C_E_s = [6088.20, 5205.72]
C_E_u = [67.825, 64.633]
Q_E_min = [0, 42000000]
Q_E_max = [42000000, 77000000]

d = 0.05 / 100

# Define the decision variables
model.x_A = p

### Code Executability

In [12]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
Q = 239600480

C_A_s = 3855.34
C_A_u = 61.150
Q_A_max = 33000000

C_B_s = [3987.26, 2887.65, 2312.94, 1967.37]
C_B_u = [63.143, 61.949, 61.356, 61.041]
Q_B_min = [0, 12000000, 30000000, 60000000]
Q_B_max = [12000000, 30000000, 60000000, 120000000]

C_C_s = 13456.00
C_C_u = 62.019
Q_C_max = 165600000

C_D_s = 6583.98
C_D_u = 72.488
Q_D_max = 12000000

C_E_s = [6088.20, 5205.72]
C_E_u = [67.825, 64.633]
Q_E_min = [0, 42000000]
Q_E_max = [42000000, 77000000]

d = 0.05 / 100

# Define the decision variables
model.x_A = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_A_max))
model.x_B = pyo.Var(4, within=pyo.Binary)
model.x_C = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_C_max))
model.x_D = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_D_max))
model.x_E = pyo.Var(2, within=pyo.Binary)

# Define the objective function
def cost_rule(model):
    return (
        C_A_s * model.x_A / 1000 + C_A_u * model.x_A +
        sum(C_B_s[i] * model.x_B[i] / 1000 + C_B_u[i] * model.x_B[i] for i in range(4)) +
        C_C_s * model.x_C / 1000 + C_C_u * model.x_C +
        C_D_s * model.x_D / 1000 + C_D_u * model.x_D +
        sum(C_E_s[i] * model.x_E[i] / 1000 + C_E_u[i] * model.x_E[i] for i in range(2))
    )
model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

# Define the constraints
model.constraints = pyo.ConstraintList()

model.constraints.add(model.x_A <= Q_A_max)

for i in range(4):
    model.constraints.add(model.x_B[i] * Q_B_min[i] <= model.x_B[i] * Q_B_max[i])

model.constraints.add(model.x_C <= Q_C_max)
model.constraints.add(model.x_D <= Q_D_max)

for i in range(2):
    model.constraints.add(model.x_E[i] * Q_E_min[i] <= model.x_E[i] * Q_E_max[i])

model.constraints.add(
    model.x_A + sum(model.x_B[i] * Q_B_max[i] for i in range(4)) +
    model.x_C + model.x_D + sum(model.x_E[i] * Q_E_max[i] for i in range(2)) == Q
)

model.constraints.add(model.x_B[1] <= Q_B_max[0] - Q_B_min[0])
model.constraints.add(model.x_B[2] <= Q_B_max[1] - Q_B_min[1])
model.constraints.add(model.x_B[3] <= Q_B_max[2] - Q_B_min[2])

model.constraints.add(model.x_E[1] <= Q_E_max[0] - Q_E_min[0])

# Solve the model
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# Print the results
print('Optimal cost:', pyo.value(model.cost))
print('x_A:', pyo.value(model.x_A))
print('x_B:', [pyo.value(model.x_B[i]) for i in range(4)])
print('x_C:', pyo.value(model.x_C))
print('x_D:', pyo.value(model.x_D))
print('x_E:', [pyo.value(model.x_E[i]) for i in range(2)])

TypeError: Cannot create a Set from data that does not support __contains__.  Expected set-like object supporting collections.abc.Collection interface, but received 'int'.

### Solution Correctness

In [13]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
Q = 239600480

C_A_s = 3855.34
C_A_u = 61.150
Q_A_max = 33000000

C_B_s = [3987.26, 2887.65, 2312.94, 1967.37]
C_B_u = [63.143, 61.949, 61.356, 61.041]
Q_B_min = [0, 12000000, 30000000, 60000000]
Q_B_max = [12000000, 30000000, 60000000, 120000000]

C_C_s = 13456.00
C_C_u = 62.019
Q_C_max = 165600000

C_D_s = 6583.98
C_D_u = 72.488
Q_D_max = 12000000

C_E_s = [6088.20, 5205.72]
C_E_u = [67.825, 64.633]
Q_E_min = [0, 42000000]
Q_E_max = [42000000, 77000000]

d = 0.05 / 100

# Define the decision variables
model.x_A = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_A_max))
model.x_B = pyo.Var(within=pyo.Binary)
model.x_C = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_C_max))
model.x_D = pyo.Var(within=pyo.NonNegativeIntegers, bounds=(0, Q_D_max))
model.x_E = pyo.Var(within=pyo.Binary)

# Define the objective function
def cost_rule(model):
    return (
        C_A_s * model.x_A / 1000 + C_A_u * model.x_A +
        sum(C_B_s[i] * model.x_B[i] / 1000 + C_B_u[i] * model.x_B[i] for i in range(4)) +
        C_C_s * model.x_C / 1000 + C_C_u * model.x_C +
        C_D_s * model.x_D / 1000 + C_D_u * model.x_D +
        sum(C_E_s[i] * model.x_E[i] / 1000 + C_E_u[i] * model.x_E[i] for i in range(2))
    )
model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

# Define the constraints
model.constraints = pyo.ConstraintList()

model.constraints.add(model.x_A <= Q_A_max)

for i in range(4):
    model.constraints.add(model.x_B[i] * Q_B_min[i] <= model.x_B[i] * Q_B_max[i])

model.constraints.add(model.x_C <= Q_C_max)
model.constraints.add(model.x_D <= Q_D_max)

for i in range(2):
    model.constraints.add(model.x_E[i] * Q_E_min[i] <= model.x_E[i] * Q_E_max[i])

model.constraints.add(
    model.x_A + sum(model.x_B[i] * Q_B_max[i] for i in range(4)) +
    model.x_C + model.x_D + sum(model.x_E[i] * Q_E_max[i] for i in range(2)) == Q
)

model.constraints.add(model.x_B[1] <= Q_B_max[0] - Q_B_min[0])
model.constraints.add(model.x_B[2] <= Q_B_max[1] - Q_B_min[1])
model.constraints.add(model.x_B[3] <= Q_B_max[2] - Q_B_min[2])

model.constraints.add(model.x_E[1] <= Q_E_max[0] - Q_E_min[0])

# Solve the model
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# Print the results
print('Optimal cost:', pyo.value(model.cost))
print('x_A:', pyo.value(model.x_A))
print('x_B:', [pyo.value(model.x_B[i]) for i in range(4)])
print('x_C:', pyo.value(model.x_C))
print('x_D:', pyo.value(model.x_D))
print('x_E:', [pyo.value(model.x_E[i]) for i in range(2)])

ERROR:pyomo.core:Rule failed when generating expression for Objective cost with index None:
KeyError: "Cannot treat the scalar component 'x_B' as an indexed component"
ERROR:pyomo.core:Constructing component 'cost' from data=None failed:
    KeyError: "Cannot treat the scalar component 'x_B' as an indexed component"


KeyError: "Cannot treat the scalar component 'x_B' as an indexed component"